# SQL con clase

*The single biggest challenge to learning SQL programming is unlearning procedureal programming.*
-- Joe Celko

En este notebook, exploraremos diversos estilos y prácticas para escribir consultas SQL enfocadas en el análisis de datos. Las consultas analíticas se distinguen de las consultas convencionales en que están diseñadas específicamente para trabajar sobre grandes volúmenes de datos y tienen a ser más complejas.

## Setup

Primero vamos a configurar la notebook y definir las variables principales.

Instalamos los paquetes necesarios. Este paso se puede omitir si ya lo realizó en su entorno, sin embargo si usa **Colab** va a ser necesario ejecutarlo.

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

Deploy Dash apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


Y finalmente cargamos el dataset en la base de datos

In [5]:
dataset_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/datasets/'
dataset_file = 'competencia_01_crudo.csv'

In [6]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


## WITH



Un escenario común es necesitar realizar una consulta a no a una tabla, sino a otra consulta. Una forma tradicional de hacerlo es usando es:
* Creando una tabla temporal con esa consulta
* Usando `selects from select`.

Esta necesidad es muy común, pero como todo en la vida hay peros.
- Crear una tabla en una base de datos que va a ser usada UNA SOLA VEZ ensucia los esquemas con un montón de tablas que no son útiles y además del desorden ocupan espacio innecesario
- El `select from select`, debe ser de las cosas peor vistas por los administradores de BBDD. Tienen sus razones técnicas, en general la recomendación es no usarlo a menos que uno conozca bien como funciona la base de datos por dentro, ya que hay casos donde terminan ocasionando muchos problemas

Y que hacer? bueno hay una clausula en **SQL** llamada `WITH`, que nos permite crear esas (plural) tablas para ser usadas en la **query** principal. Ejemplo si queremos comparar el valor de una variable con su promedio a través de todos los meses.


In [7]:
%%sql
with promedios as (
    select
        numero_de_cliente
        , avg(mrentabilidad) as avg_mrentabilidad
    from competencia_01_crudo
    group by numero_de_cliente
) select
    cp.numero_de_cliente
    , cp.foto_mes
    , if(cp.mrentabilidad > p.avg_mrentabilidad, 'mayor', 'menor') as comparacion
from competencia_01_crudo cp
join promedios p using (numero_de_cliente)


,numero_de_cliente,foto_mes,comparacion
0,301268134,202103,mayor
1,301272583,202103,menor
2,301274914,202103,mayor
3,301280892,202103,menor
4,301295555,202103,mayor
...,...,...,...
981941,1583776239,202106,mayor
981942,1588482879,202106,menor
981943,1589570953,202106,mayor
981944,1591572581,202106,menor


- Usar esa clausula tiene muchos beneficios, entre ellos sumar legibilidad de código y le permite al motor de **SQL** optimizar mejor, ya que entiende todo lo que se buscar hacer y planifica como resolverlo en el menor tiempo posible.

**NOTA**: Hay un tipo de tabla que se llaman temporales, que se borran solas una vez que uno se desconecta de la base de datos. Es una mejor alternativa frente al `CREATE`, sin embargo si la tabla que se esta creando se usa una sola vez es conveniente usar el `WITH` porque permite optimizar la ejecución. Si esa tabla va a ser usada mas de una vez dentro del proceso, las tablas temporales son la mejor opción.

# Funciones Analíticas

Las funciones analíticas en SQL son un conjunto de funciones que te permiten realizar cálculos avanzados sobre un conjunto de filas relacionadas dentro de una consulta conservando las filas individuales y calculando sobre ventanas de datos relacionadas.

Veamos un ejemplo para que quede más claro:

In [8]:
%%sql
select
    numero_de_cliente
    , foto_mes
    , mrentabilidad
from competencia_01_crudo
where numero_de_cliente = 249221109
order by foto_mes

,numero_de_cliente,foto_mes,mrentabilidad
0,249221109,202101,1664.94
1,249221109,202102,1002.92
2,249221109,202103,769.74
3,249221109,202104,1532.79
4,249221109,202105,1804.14
5,249221109,202106,399.88


In [9]:
%%sql
select
    numero_de_cliente
    , foto_mes
    , mrentabilidad
    , lead(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_mas_1
    , lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_1
    , lead(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_mas_2
    , lag(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_2
from competencia_01_crudo
where numero_de_cliente = 249221109
order by foto_mes



,numero_de_cliente,foto_mes,mrentabilidad,mrentabilidad_mas_1,mrentabilidad_menos_1,mrentabilidad_mas_2,mrentabilidad_menos_2
0,249221109,202101,1664.94,1002.92,NaN,769.74,NaN
1,249221109,202102,1002.92,769.74,1664.94,1532.79,NaN
2,249221109,202103,769.74,1532.79,1002.92,1804.14,1664.94
3,249221109,202104,1532.79,1804.14,769.74,399.88,1002.92
4,249221109,202105,1804.14,399.88,1532.79,NaN,769.74
5,249221109,202106,399.88,NaN,1804.14,NaN,1532.79


* Qué paso?
* ¿Cómo esta construyendo las nuevas variables?
* ¿ Nos puede ayudar a construir targets ?

¿Dondé puedo leer un poco más de esta magia negra? https://duckdb.org/docs/sql/window_functions.html

## Los casos raros

Que pasa con el cliente **304051318** y la foto **202103**


In [10]:
%%sql
select
    numero_de_cliente
    , foto_mes
from competencia_01_crudo
where numero_de_cliente = 304051318
order by foto_mes

,numero_de_cliente,foto_mes
0,304051318,202101
1,304051318,202102
2,304051318,202104
3,304051318,202105
4,304051318,202106


* ¿Cuál debería ser la clase para cada periodo?

### Una pista

Podemos generar todas las posibles combinaciones de clientes y periodos de manera muy simple

In [11]:
%%sql
with periodos as (
  select distinct foto_mes from competencia_01_crudo
), clientes as (
  select distinct numero_de_cliente from competencia_01_crudo
)
select numero_de_cliente, foto_mes from clientes cross join periodos
where numero_de_cliente = 304051318
order by foto_mes

,numero_de_cliente,foto_mes
0,304051318,202101
1,304051318,202102
2,304051318,202103
3,304051318,202104
4,304051318,202105
5,304051318,202106


Solo nos queda saber si estuvo o no el banco el cliente en ese periodo y armar el

In [12]:
%%sql
with periodos as (
    select distinct foto_mes from competencia_01_crudo -- Esto también se puede hacer con secuencias
), clientes as (
    select distinct numero_de_cliente from competencia_01_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
)
select
    t.numero_de_cliente
    , t.foto_mes
    , if(c.numero_de_cliente is null, 0, 1) as mes_0
from todo t
left join competencia_01_crudo c using (numero_de_cliente, foto_mes)
where t.numero_de_cliente = 304051318
order by foto_mes

,numero_de_cliente,foto_mes,mes_0
0,304051318,202101,1
1,304051318,202102,1
2,304051318,202103,0
3,304051318,202104,1
4,304051318,202105,1
5,304051318,202106,1


Antes de continuar, tomémonos un momento para reflexionar. Con todas estas piezas, ¿cómo podemos ensamblar el target?

Una vez que lo tengamos claro sobre el papel, proceda a completar el código que sigue.

In [15]:
%%sql
create or replace table competencia_01 as
with periodos as (
  select distinct foto_mes from competencia_01_crudo
), clientes as (
  select distinct numero_de_cliente from competencia_01_crudo
), todo as (
  select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
  select
  c.*
    , if(c.numero_de_cliente is null, 0, 1) as mes_0
  , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
  , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
  , case 
      when mes_2 = 1 then 'CONTINUA'
      when mes_1 = 1 and mes_2 = 0 then 'BAJA+2'
      when mes_1 = 0 and mes_2 = 0 then 'BAJA+1'
      when mes_1 = 0 and mes_2 = 1 then 'CONTINUA'
      else null
    end as clase_ternaria
  from todo t
  left join competencia_01_crudo c using (numero_de_cliente, foto_mes)
) 
select
* EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1

,Success


Revisamos que todo salga como esperamos

In [16]:
%sql select count(*) from competencia_01

,count_star()
0,981946


In [17]:
%sql select * from competencia_01 LIMIT 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249619949,202101,1,0,0,45,68,-4732.12,-19793.21,295.29,...,4,0.00,-15639.77,0.0,1634,1425.73,3,0,5947.11,CONTINUA
1,249619949,202102,1,0,0,45,69,-4565.57,-21930.03,681.57,...,-3,28067.15,-28067.15,0.0,1662,1629.77,1,0,6944.16,CONTINUA
2,249619949,202103,1,0,0,45,70,-3993.18,-23437.50,1881.56,...,1,0.00,-28067.15,0.0,1693,48114.51,6,0,11600.97,CONTINUA
3,249619949,202104,1,0,0,45,71,-4296.05,-26240.03,3213.29,...,2,0.00,-92740.78,0.0,1723,4767.54,3,0,7542.39,CONTINUA
4,249619949,202105,1,0,0,45,72,-8159.42,-33129.26,837.17,...,5,0.00,-40819.81,0.0,1754,0.00,0,0,5794.62,None


In [18]:

%sql df << select * from competencia_01

Y vemos la cardinalidad de las clases por periodo

In [20]:
df_sorted[df_sorted["numero_de_cliente"]==304051318]

,numero_de_cliente,foto_mes,diff_meses
46513,304051318,2021-01-01,NaN
46514,304051318,2021-02-01,1.018397
46515,304051318,2021-04-01,1.938239
46516,304051318,2021-05-01,0.985545
46517,304051318,2021-06-01,1.018397


In [124]:
df2= df[["numero_de_cliente","foto_mes"]]

# Convertimos 'foto_mes' a string para asegurar el formato correcto
df2['foto_mes'] = df2['foto_mes'].astype(str)

# Convertimos 'foto_mes' a un objeto datetime, especificando el formato YYYYMM
df2['foto_mes'] = pd.to_datetime(df2['foto_mes'], format='%Y%m')

df_sorted = df2.sort_values(['numero_de_cliente', 'foto_mes'])

# Calcula la diferencia en meses
df_sorted['diff_meses'] = df_sorted.groupby('numero_de_cliente')['foto_mes'].diff().dt.days / 30.44

# Identifica los clientes que se fueron un mes y volvieron
clientes_retornados = df_sorted[df_sorted['diff_meses'] > 1.5][['numero_de_cliente', 'foto_mes']]

# Muestra los resultados
print(clientes_retornados)

        numero_de_cliente   foto_mes
123139          263751894 2021-06-01
650371          304051318 2021-04-01
134291          351895410 2021-05-01
311198          445822700 2021-04-01
452779          493399293 2021-05-01
188774          555298549 2021-05-01
937862          559063769 2021-06-01
636418          583196648 2021-06-01
385122          644687585 2021-03-01
505664          675442310 2021-06-01
19850           832767503 2021-04-01
955996          855499233 2021-06-01
481191          885810926 2021-06-01
35629           893574159 2021-06-01
925617          933210746 2021-04-01
216296         1179226843 2021-05-01
641416         1302790375 2021-04-01
810115         1389447490 2021-05-01
381480         1435040696 2021-06-01
122769         1460648156 2021-05-01
600894         1471746344 2021-06-01
904934         1524390423 2021-05-01
787513         1549075943 2021-04-01


In [21]:
%%sql
PIVOT competencia_01
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,202101,635,785,160606
1,202102,785,1017,160844
2,202103,1020,981,161684
3,202104,982,1189,161919
4,202105,0,0,0
5,202106,0,0,0


Y finalmente almacenamos el nuevo dataset para ser usado a continuación

In [22]:
%%sql 
COPY competencia_01 TO '{{dataset_path}}competencia_01.csv' (FORMAT CSV, HEADER)

,Success


In [13]:
%%sql
select foto_mes,clase_ternaria from competencia_01 c
where c.numero_de_cliente = 304051318
order by foto_mes

,foto_mes,clase_ternaria
0,202101,BAJA+2
1,202102,CONTINUA
2,202104,CONTINUA
3,202105,None
4,202106,None



## El pato del amor

DuckDB no es una base de datos SQL estándar; su sintaxis moderna facilita significativamente la escritura de consultas. A continuación, te menciono algunas características destacadas:

+ Uso de variables: En DuckDB, no es necesario reescribir las variables; puedes utilizar tus variables una vez definidas, lo que simplifica el código.

+ Función IF: DuckDB soporta la función IF(condición, valor_si_verdadero, valor_si_falso), lo que hace que tu código sea más claro y legible en comparación con el uso de CASE WHEN.

+ Exclusión de campos: Puedes utilizar SELECT * EXCLUDE(field1, ...) para excluir campos específicos de la lista de selección, lo que ahorra tiempo y esfuerzo.

+ Alias en cláusulas: Es posible utilizar alias en las cláusulas WHERE, GROUP BY y HAVING, lo que añade flexibilidad a la escritura de consultas.

+ Uso de alias en cálculos: DuckDB permite el uso de alias dentro del cálculo de otros campos, facilitando la manipulación y transformación de datos en una consulta.

Estas son solo algunas de las capacidades que hacen de DuckDB una herramienta poderosa. Te recomiendo leer la documentación y el blog oficial para explorar más a fondo su potencial.